# EduAgent20250707


## 并行推进三个模块：题库，批改，报告

# 题库：实现一个从试题图片到录入数据库的工作流

### PAPER

| 字段名                         | 类型             | 描述                                      |
|------------------------------|------------------|-------------------------------------------|
|paper_id|primary key|时间戳加随机数生成，保证唯一性|
|paper_name|string|试卷名称，带有地区或全国卷类型属性。例：2024年普通高中学业水平选择性考试（湖北卷）；2024年普通高等学校招生全国统一考试（全国）|
|paper_auxiliary_name|string|试卷辅助名称，例：理综化学部分
|paper_meta_data_description|string|试卷元数据描述，例：本试卷满分150分，考试时间120分钟|
|total_score|int|本试卷满分多少分|
|time_limit|int|考试时间|
|question_number_sequence_starts_at|int|本试卷阿拉伯数字标号的题目从哪道题开始|
|question_number_sequence_ends_at|int|本试卷阿拉伯数字标号的题目从哪道题结束|
|subject|string|所属科目|
|year|int|年份|

### SECTION

| 字段名                         | 类型             | 描述                                      |
|------------------------------|------------------|-------------------------------------------|
|paper_it_belongs_to_id|foreign key --> paper_id from PAPER|所属试卷的唯一标识|
|section_id|int|sha256生成，保证唯一性|
|section_meta_data_description|string|本部分元数据描述，例：本题共5小题，共77分。解答应写出文字说明、证明过程或演算步骤|
|total_score|int|本部分满分多少分|
|question_number_sequence_starts_at|int|本部分阿拉伯数字标号的题目从哪道题开始|
|question_number_sequence_ends_at|int|本部分阿拉伯数字标号的题目从哪道题结束|
|section_name|string|本部分名称，例；一、选择题|

（paepr_it_belongs_to_id, section_id） = primary key

### QUESTION

| 字段名                         | 类型             | 描述                                      |
|------------------------------|------------------|-------------------------------------------|
| question_id                  | primary key      | 随机生成，保证唯一性                      |
| paper_it_belongs_to          | foreign key --> paper_id from PAPER| 链接PAPER
| section_it_belongs_to        | foreign key --> (paepr_it_belongs_to_id, section_id) from SECTION|链接SECTION
| question_number              | number       | 题号                                      |
| question_text         | text             | 题目正文                                  |
| question_markdown | text |可以直接被渲染的对象
| question_text_embedding      | str = json.dumps(list[float])      | 问题文字的嵌入向量                        |
| score_of_question            | int            | 这道题的分值                                  |
| question_answer              | text             | 答案内容                                  |   
| question_focus     |text|照抄命题点|




## 完美适用于高考真题录入👍 
## ！⚠️未来需要拓展适配零散单题录入⚠️！

## 创立数据库

In [72]:
import sqlite3

# 创建数据库连接
conn = sqlite3.connect('edu_agent_database.db')
cursor = conn.cursor()

print("数据库连接已建立")
# ==============================================================================

# 创建PAPER表
create_paper_table = """
CREATE TABLE IF NOT EXISTS PAPER (
    paper_id VARCHAR(100) PRIMARY KEY,
    paper_name VARCHAR(100) NOT NULL,
    paper_auxiliary_name VARCHAR(100) NOT NULL,
    paper_meta_data_description TEXT,
    total_score INTEGER NOT NULL,
    time_limit INTEGER NOT NULL,
    question_number_sequence_starts_at INTEGER NOT NULL,
    question_number_sequence_ends_at INTEGER NOT NULL,
    subject VARCHAR(50) NOT NULL,
    year INTEGER NOT NULL
);
"""

cursor.execute(create_paper_table)
print("PAPER表创建成功")

# ==============================================================================

# 创建SECTION表
create_section_table = """
CREATE TABLE IF NOT EXISTS SECTION (
    paper_it_belongs_to_id VARCHAR(50) NOT NULL,
    section_id INTEGER NOT NULL,
    section_meta_data_description TEXT,
    total_score INTEGER NOT NULL,
    question_number_sequence_starts_at INTEGER NOT NULL,
    question_number_sequence_ends_at INTEGER NOT NULL,
    section_name VARCHAR(200) NOT NULL,
    PRIMARY KEY (paper_it_belongs_to_id, section_id),
    FOREIGN KEY (paper_it_belongs_to_id) REFERENCES PAPER(paper_id) ON DELETE CASCADE
);
"""

cursor.execute(create_section_table)
print("SECTION表创建成功")

# ==============================================================================

# 创建QUESTION表
create_question_table = """
CREATE TABLE IF NOT EXISTS QUESTION (
    question_id VARCHAR(50) PRIMARY KEY,
    paper_it_belongs_to_id VARCHAR(50) NOT NULL,
    section_it_belongs_to_id INTEGER NOT NULL,
    question_number VARCHAR(20) NOT NULL,
    question_text TEXT NOT NULL,
    question_text_embedding TEXT, 
    question_markdown TEXT, --可以直接被渲染的对象
    score_of_question REAL NOT NULL,
    question_answer TEXT NOT NULL,
    question_focus TEXT, -- 命题点
    FOREIGN KEY (paper_it_belongs_to_id) REFERENCES PAPER(paper_id) ON DELETE CASCADE,
    FOREIGN KEY (paper_it_belongs_to_id, section_it_belongs_to_id) 
        REFERENCES SECTION(paper_it_belongs_to_id, section_id) ON DELETE CASCADE
);
"""

cursor.execute(create_question_table)
print("QUESTION表创建成功")


PAPER表创建成功
SECTION表创建成功
QUESTION表创建成功


## 插入PAPER数据

In [ ]:
import hashlib
import datetime
import random

# 获取当前时间，格式化为字符串
now_str = datetime.datetime.now().strftime("%Y%m%d%H%M%S")  # 例如 20240630153045

# 生成一个随机整数（比如5位数）
rand_num = random.randint(10000, 99999)  # 例如 48392

# 拼接成一个字符串
input = f"{now_str}_{rand_num}"

print(input)

def generate_id(input: str) -> str:
    """
    根据当前时间戳和随机数生成唯一且加密的id。
    """
    hash_object = hashlib.sha256(input.encode('utf-8'))
    id = hash_object.hexdigest()  # 64位16进制字符串
    return id

# 示例
paper_id = generate_id(input)
print(paper_id)

import os

subject_pinyin = "shengwu"
os.system(f"mkdir pic/{subject_pinyin}")
os.system(f"mkdir pic/{subject_pinyin}/{paper_id}")

paper_pic_path = f"pic/{subject_pinyin}/{paper_id}/paper.png"

20250705121313_87309
a9b2fa894481ff63b366133f56fb4986e07091aa9be69cc51c55d41721a75c79


mkdir: pic/shengwu2: File exists


# ！！在运行下面代码块之前， 检查paper.png是否放进来了！！

In [73]:
from pydantic import BaseModel, Field
from enum import Enum
from typing import List, Optional, Literal
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()

# 定义结构化输出的数据模型
class SubjectEnum(str, Enum):
    语文 = "语文"
    数学 = "数学"
    英语 = "英语"
    物理 = "物理"
    化学 = "化学"
    生物 = "生物"
    政治 = "思想政治"
    历史 = "历史"
    地理 = "地理"

class PAPER(BaseModel):
    paper_name: str = Field(..., example="2024年普通高中学业水平选择性考试（湖北卷）")
    paper_auxiliary_name: str = Field(..., example="生物学",description="试卷的辅助名称，在图片中位于试卷名称下方和试卷元数据描述上方")
    paper_meta_data_description: str = Field(..., example="本试卷共100分，考试时间75分钟")
    total_score: int = Field(..., example=100)
    time_limit: int = Field(..., example=75)
    subject: SubjectEnum = Field(..., example="生物")
    year: int = Field(..., example=2024)



# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
file_id = create_file(paper_pic_path)

response_paper = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "提取试卷的基本信息"},
            {
                "type": "input_image",
                "file_id": file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=PAPER,
)

print(response_paper.output_parsed)



paper_name='2024年普通高中学业水平选择性考试（湖北卷）' paper_auxiliary_name='生物学' paper_meta_data_description='本试卷共100分，考试时间75分钟' total_score=100 time_limit=75 subject=<SubjectEnum.生物: '生物'> year=2024


In [75]:
import sqlite3
from typing import Optional


db_path = "edu_agent_database.db"
paper_id = paper_id
paper_name = response_paper.output_parsed.paper_name
paper_auxiliary_name = response_paper.output_parsed.paper_auxiliary_name
paper_meta_data_description = response_paper.output_parsed.paper_meta_data_description
total_score = response_paper.output_parsed.total_score
time_limit = response_paper.output_parsed.time_limit
question_number_sequence_starts_at = 1 #手动输入
question_number_sequence_ends_at = 18 #手动输入
subject = response_paper.output_parsed.subject
year = response_paper.output_parsed.year


def insert_paper(
    db_path: str,
    paper_id: str,
    paper_name: str,
    paper_auxiliary_name: Optional[str] = None,
    paper_meta_data_description: Optional[str] = None,
    total_score: int = 100,
    time_limit: int = 120,  # 分钟
    question_number_sequence_starts_at: int = 1,
    question_number_sequence_ends_at: int = 22,
    subject: str = "",
    year: int = 2024
):
    """
    向PAPER表插入一条记录
    
    参数:
    - db_path: 数据库文件路径
    - paper_id: 试卷ID
    - paper_name: 试卷名称
    - paper_auxiliary_name: 试卷副标题/辅助名称
    - paper_meta_data_description: 试卷元数据描述
    - total_score: 总分
    - time_limit: 时间限制（分钟）
    - question_number_sequence_starts_at: 从哪道题开始 
    - question_number_sequence_ends_at: 从哪道题结束
    - subject: 科目
    - year: 年份
    """
    
    # SQL插入语句
    sql = """
    INSERT INTO PAPER (
        paper_id, 
        paper_name, 
        paper_auxiliary_name, 
        paper_meta_data_description, 
        total_score, 
        time_limit, 
        question_number_sequence_starts_at,
        question_number_sequence_ends_at, 
        subject, 
        year
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    try:
        # 连接数据库
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # 执行插入操作
        cursor.execute(sql, (
            paper_id,
            paper_name,
            paper_auxiliary_name,
            paper_meta_data_description,
            total_score,
            time_limit,
            question_number_sequence_starts_at,
            question_number_sequence_ends_at,
            subject,
            year
        ))
        
        # 提交事务
        conn.commit()
        print(f"成功插入试卷记录：{paper_name} (ID: {paper_id})")
        
    except sqlite3.Error as e:
        print(f"数据库操作失败: {e}")
        conn.rollback()
        
    except Exception as e:
        print(f"发生错误: {e}")
        
    finally:
        # 关闭连接
        if conn:
            conn.close()

insert_paper(db_path, paper_id, paper_name, paper_auxiliary_name, paper_meta_data_description, total_score, time_limit, question_number_sequence_starts_at, question_number_sequence_ends_at, subject, year)

成功插入试卷记录：2024年普通高中学业水平选择性考试（湖北卷） (ID: 001c825dc71cd3e95b796cd8125f9ac4ad7736302bc8108e1df1345124edf358)


## 插入SECTION数据

In [5]:
import hashlib
import datetime
import random

# 获取当前时间，格式化为字符串
now_str = datetime.datetime.now().strftime("%Y%m%d%H%M%S")  # 例如 20240630153045

# 生成一个随机整数（比如5位数）
rand_num = random.randint(10000, 99999)  # 例如 48392

# 拼接成一个字符串
input = f"{now_str}_{rand_num}"

print(input)

def generate_id(input: str) -> str:
    """
    根据当前时间戳和随机数生成唯一且加密的id。
    """
    hash_object = hashlib.sha256(input.encode('utf-8'))
    id = hash_object.hexdigest()  # 64位16进制字符串
    return id

# 示例
section_id = generate_id(input)
print(section_id)

import os


os.system(f"mkdir pic/{subject_pinyin}/{paper_id}/{section_id}")

section_pic_path = f"pic/{subject_pinyin}/{paper_id}/{section_id}/section.png"

20250704231417_33734
a1890557bd7495f1c4e40edb274c289362aa472b82c0a8c903b07c7164dfa31a


# ！！在运行下面代码块之前， 检查section.png是否放进来了！！

In [94]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()
# 定义结构化输出的数据模型
class SECTION(BaseModel):
    section_name: str = Field(..., example="一、选择题")
    section_meta_data_description: str = Field(..., example="本题共5小题，共77分。解答应写出文字说明、证明过程或演算步骤")
    total_score: int = Field(..., example=44)

# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
file_id = create_file(section_pic_path)

response_section = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "提取试卷该部分的基本信息"},
            {
                "type": "input_image",
                "file_id": file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=SECTION,
)

print(response_section.output_parsed)



section_name='一、选择题' section_meta_data_description='本题共18小题，每小题2分，共36分。在每小题给出的四个选项中，只有一项是符合题目要求的。' total_score=36


In [ ]:
import sqlite3
from typing import Optional

db_path="edu_agent_database.db"
paper_it_belongs_to_id= paper_id
section_id = section_id
section_name = response_section.output_parsed.section_name
section_meta_data_description = response_section.output_parsed.section_meta_data_description
total_score = response_section.output_parsed.total_score
question_number_sequence_starts_at = 1 #手动输入
question_number_sequence_ends_at = 18 #手动输入


def insert_section(
    db_path: str,
    paper_it_belongs_to_id: str,
    section_name: str,
    section_meta_data_description: Optional[str] = None,
    total_score: int = 0,
    question_number_sequence_starts_at: int = 1,
    question_number_sequence_ends_at: int = 1,
    section_id: Optional[int] = None
):
    """
    向SECTION表插入一条记录
    
    参数:
    - db_path: 数据库文件路径
    - paper_it_belongs_to_id: 所属试卷ID（外键）
    - section_name: 部分名称，例如"一、选择题"
    - section_meta_data_description: 部分元数据描述
    - total_score: 本部分满分
    - question_number_sequence_starts_at: 题目编号开始位置
    - question_number_sequence_ends_at: 题目编号结束位置
    - section_id: 部分ID
    """
    

    
    # SQL插入语句
    sql = """
    INSERT INTO SECTION (
        paper_it_belongs_to_id,
        section_id,
        section_meta_data_description,
        total_score,
        question_number_sequence_starts_at,
        question_number_sequence_ends_at,
        section_name
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    
    try:
        # 连接数据库
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # 执行插入操作
        cursor.execute(sql, (
            paper_it_belongs_to_id,
            section_id,
            section_meta_data_description,
            total_score,
            question_number_sequence_starts_at,
            question_number_sequence_ends_at,
            section_name
        ))
        
        # 提交事务
        conn.commit()
        print(f"成功插入试卷部分：{section_name} (ID: {section_id}, 试卷: {paper_it_belongs_to_id})")
        
        return section_id
        
    except sqlite3.IntegrityError as e:
        print(f"数据完整性错误 - 可能存在重复记录或外键约束问题: {e}")
        conn.rollback()
        return None
        
    except sqlite3.Error as e:
        print(f"数据库操作失败: {e}")
        conn.rollback()
        return None
        
    except Exception as e:
        print(f"发生错误: {e}")
        return None
        
    finally:
        # 关闭连接
        if conn:
            conn.close()

insert_section(db_path,paper_it_belongs_to_id,section_name,section_meta_data_description,total_score,question_number_sequence_starts_at,question_number_sequence_ends_at,section_id)

## 插入QUESTION数据

In [59]:
import hashlib
import datetime
import random

# 获取当前时间，格式化为字符串
now_str = datetime.datetime.now().strftime("%Y%m%d%H%M%S")  # 例如 20240630153045

# 生成一个随机整数（比如5位数）
rand_num = random.randint(10000, 99999)  # 例如 48392

# 拼接成一个字符串
input = f"{now_str}_{rand_num}"

print(input)

def generate_id(input: str) -> str:
    """
    根据当前时间戳和随机数生成唯一且加密的 question_id。
    """
    hash_object = hashlib.sha256(input.encode('utf-8'))
    id = hash_object.hexdigest()  # 64位16进制字符串
    return id

# 示例
question_id = generate_id(input)
print(question_id)

20250703164853_15109
759a77c59f8b91282283f54dbb0529b9ef1ea5b0db4e0288e42ba9f9e97f067b


In [63]:
import os

os.system(f"mkdir pic/{subject_pinyin}/{paper_id}/{section_id}/{question_id}")

question_image_annotated_pic_path = f"pic/{subject_pinyin}/{paper_id}/{section_id}/{question_id}/question_image_annotated.png"
answer_and_question_focus_pic_path = f"pic/{subject_pinyin}/{paper_id}/{section_id}/{question_id}/answer_and_question_focus.png"

mkdir: pic/759a77c59f8b91282283f54dbb0529b9ef1ea5b0db4e0288e42ba9f9e97f067b: File exists


# ！！在运行下一个代码块之前，检查question_image_annotated.png, answer_and_question_focus.png, question_image.png, answer_explanation.png，figure_1... 是否放进来了！！ 

In [57]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()
# 定义结构化输出的数据模型
class QUESTION_question_markdown(BaseModel):
    question_markdown: str = Field(..., example = "", description="将这道题所有的文字包括表格转化为markdown格式，直接能被渲染那种，换行用<br>。每个红色方框代表一个图片位置，按先后顺序给每个图片位置插入相对路径，例如: ![图片描述](/1.png); ![图片描述](/2.png)....")
   
class QUESTION_question_text(BaseModel):
   question_text: str = Field(..., example = "", description="将这道题所有的文字包括表格转化为markdown格式。遇到有图片的地方，只保留图片描述即可")
        

class QUESTION_answer_part(BaseModel):
    question_answer: str = Field(..., example="")
    #question_answer_explanation: str = Field(..., example="", description="将答案深度解析所有的文字包括表格转化为markdown格式。遇到有图片的地方，保留图片描述即可。")
    question_focus: str = Field(..., example="", description="照抄“命题点”")

# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
question_file_id = create_file(question_image_annotated_pic_path)
answer_file_id = create_file(answer_and_question_focus_pic_path)

response_question_markdown = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "你现在是一个扫描识别人，我会给你一个题目的扫描件，你需要将红框框住以外的文字正确识别，对于红框部分则不用识别。识别过程中如果检测到需要换行的话需要遵循markdown格式，markdown格式的换行符是<br>,而不是/n或者/n/n，我不想看到任何/n。每个红色方框代表一个图片位置，按先后顺序在识别出来的文字中插入代表红框序列的数字的相对路劲，例如: 第一个红框就不用识别文字，只要打印![图片描述](figure_1.png)即可;第二个红框也是 ![图片描述](figure_2.png)...."},
            {
                "type": "input_image",
                "file_id": question_file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=QUESTION_question_markdown,
)
print(response_question_markdown.output_parsed)

response_question_text = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "你现在是一个扫描识别人，我会给你一个题目的扫描件，你需要将红框框住以外的文字正确识别，每个红色方框代表一个图片位置，请你描述这个图片。识别过程中如果检测到需要换行的话需要遵循markdown格式，markdown格式的换行符是<br>,而不是/n或者/n/n，我不想看到任何/n。"},
            {
                "type": "input_image",
                "file_id": question_file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=QUESTION_question_text,
)

print(response_question_text.output_parsed)

response_answer_part = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "你现在是一个扫描识别人，我会给你一个题目答案的扫描件，请按照要求提取question_answer（题目的答案不包括【命题点】）和question_focus（即【命题点】后面的内容）。和识别过程中如果检测到需要换行的话需要遵循markdown格式，markdown格式的换行符是<br>,而不是/n或者/n/n，我不想看到任何/n。"},
            {
                "type": "input_image",
                "file_id": answer_file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=QUESTION_answer_part,
)

print(response_answer_part.output_parsed)



question_answer='(1) ②<br>(2) ③ 系谱图③中I代双亲均不患病，但II代女儿患病，说明该病为常染色体隐性遗传病，而S病为常染色体显性遗传病，故③一定不属于S病 ①②<br>(3) 不能。乙(1号)为杂合子，2号和4号为纯合子，由于无法判断乙(1号)致病基因对应的条带，所以不能确定2号和4号是否携带致病基因<br>(4) 若该类型疾病女性患者为杂合子，在减数分裂I过程中致病基因和正常基因随同源染色体的分离而分开' question_focus='遗传病的检测和预防、遗传系谱图的判定、减数分裂'


In [ ]:
import sqlite3
import json
from typing import Optional
import ollama

db_path="/Users/chizhongwang/edu_agent/edu_agent_database.db"
question_id = question_id
paper_it_belongs_to_id = paper_id
section_it_belongs_to_id = section_id
question_number = 1 # 手动输入
score_of_question = 4 # 手动输入
question_text = response_question_text.output_parsed.question_text
question_markdown = response_question_markdown.output_parsed.question_markdown
question_text_embedding = json.dumps(ollama.embed(
  model="mxbai-embed-large",
  input=question_text
)["embeddings"][0])
question_answer = response_answer_part.output_parsed.question_answer
question_focus = response_answer_part.output_parsed.question_focus



def insert_question(
    db_path: str,
    question_id: str,
    paper_it_belongs_to_id: str,
    section_it_belongs_to_id: str,
    question_number: int,
    score_of_question: int,
    question_text: str,
    question_markdown: str,
    question_text_embedding: str,
    question_answer: str,
    question_focus: str,
):
    """
    向QUESTION表插入一条记录
    
    参数:
    - db_path: 数据库文件路径
    - question_id: str
    - paper_it_belongs_to_id: 所属试卷ID（外键）
    - section_it_belongs_to_id: 所属部分ID（外键）
    - question_number: int
    - score_of_question: int
    - question_text: str
    - question_markdown: str
    - question_text_embedding: str
    - question_answer: str
    - question_focus: str
    """
    
    # SQL插入语句
    sql = """
    INSERT INTO QUESTION (
        question_id,
        paper_it_belongs_to_id,
        section_it_belongs_to_id,
        question_number,
        score_of_question,
        question_text,
        question_markdown,
        question_text_embedding,
        question_answer,
        question_focus,
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    try:
        # 连接数据库
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # 执行插入操作
        cursor.execute(sql, (
            question_id,
            paper_it_belongs_to_id,
            section_it_belongs_to_id,
            question_number,
            score_of_question,
            question_text,
            question_markdown,
            question_text_embedding,
            question_answer,
            question_focus,
        ))
        
        # 提交事务
        conn.commit()
        print(f"成功插入试卷部分：{section_name} (ID: {section_id}, 试卷: {paper_it_belongs_to_id})")
        
        return section_id
        
    except sqlite3.IntegrityError as e:
        print(f"数据完整性错误 - 可能存在重复记录或外键约束问题: {e}")
        conn.rollback()
        return None
        
    except sqlite3.Error as e:
        print(f"数据库操作失败: {e}")
        conn.rollback()
        return None
        
    except Exception as e:
        print(f"发生错误: {e}")
        return None
        
    finally:
        # 关闭连接
        if conn:
            conn.close()

insert_question(db_path,
                question_id,
                paper_it_belongs_to_id,
                section_it_belongs_to_id,
                question_number,
                score_of_question,
                question_text,
                question_markdown,
                question_text_embedding,
                question_answer,
                question_focus,
                )

## 目前的录入效率：对于熟练操作者，两小时一张22道题的卷子，大约六分钟一道题
## 如何提效：⚠️在不必要的环节用图片ocr来替代调用llm

## 合并图片用下面代码

In [1]:
import os
from PIL import Image

def merge_images_vertically(image_folder, output_path="merged_output.png"):
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    image_files.sort()
    images = [Image.open(os.path.join(image_folder, f)) for f in image_files]

    max_width = max(img.width for img in images)
    resized_images = []
    for img in images:
        if img.width != max_width:
            new_height = int(img.height * (max_width / img.width))
            img = img.resize((max_width, new_height), Image.Resampling.LANCZOS)
        resized_images.append(img)

    total_height = sum(img.height for img in resized_images)
    merged_img = Image.new('RGB', (max_width, total_height), color=(255,255,255))

    y_offset = 0
    for img in resized_images:
        merged_img.paste(img, (0, y_offset))
        y_offset += img.height

    merged_img.save(output_path)
    print(f"合成图片已保存到: {output_path}")



# 用法示例
merge_images_vertically("/Users/chizhongwang/edu_agent/lishi_17_(3)_highlighted", "lishi_17_(3)_auxiliary_highlight.png")

合成图片已保存到: lishi_17_(3)_auxiliary_highlight.png
